In [1]:
from sympy import symbols, exp, Matrix, Function, Eq, oo
import os
from pathlib import Path
import DifferentiableStateSpaceModels
from DifferentiableStateSpaceModels import *
import pandas as pd

In [2]:
# Define the symbols
α, β, ρ, δ, σ, Ω_1 = symbols('α β ρ δ σ Ω_1')
t = symbols('t', integer=True)
k, z, c, q = symbols('k z c q', cls=Function)

# Define the states, controls, and parameters
x = [k, z]  # states
y = [c, q]  # controls
p = [α, β, ρ, δ, σ, Ω_1]  # parameters

# Define the system of model equations
H = [1 / c(t) - (β / c(t+1)) * (α * exp(z(t+1)) * k(t+1)**(α - 1) + (1 - δ)),
     c(t) + k(t+1) - (1 - δ) * k(t) - q(t),
     q(t) - exp(z(t)) * k(t)**α,
     z(t+1) - ρ * z(t)]

# Define the steady states
steady_states = [Eq(k(oo), (((1 / β) - 1 + δ) / α)**(1 / (α - 1))),
                 Eq(z(oo), 0),
                 Eq(c(oo), (((1 / β) - 1 + δ) / α)**(α / (α - 1)) -
                    δ * (((1 / β) - 1 + δ) / α)**(1 / (α - 1))),
                 Eq(q(oo), (((1 / β) - 1 + δ) / α)**(α / (α - 1)))]

# Define the matrix for the 1 shock
Γ = Matrix([σ])

# Define the n_x * n_ϵ matrix
η = Matrix([0, -1])

# Define the observation matrix
Q = Matrix([[1.0, 0, 0, 0],
            [0, 0, 1.0, 0]])

# Define the diagonal cholesky of covariance matrix for observation noise
Ω = [Ω_1, Ω_1]

# Generates the files and includes if required.  If the model is already created, then just loads
overwrite_model_cache  = True

In [3]:
def default_model_cache_location():
    """
    Returns the default location for the model cache
    :return: path of a folder joining the package directory and ".function_cache"
    """
    return os.path.join(Path(DifferentiableStateSpaceModels.__file__).parent.absolute(), ".function_cache")

In [4]:
default_model_cache_location()

'/Users/zhenqihu/Projects/Childers et al. (2022)/TSRL-Internship-Exercise/DifferentiableStateSpaceModels/.function_cache'

In [5]:
model_cache_location = default_model_cache_location()
model_name = "RBC"
# path of the module to be saved in the cache
module_cache_path = os.path.join(model_cache_location, model_name + ".py")

In [6]:
module_cache_path

'/Users/zhenqihu/Projects/Childers et al. (2022)/TSRL-Internship-Exercise/DifferentiableStateSpaceModels/.function_cache/RBC.py'

In [7]:
print_level = 1
# only load cache if the module isn't already loaded in memory
if (str(model_name) in globals()) and (not overwrite_model_cache):
    if print_level > 0:
        print(f"Using existing module {model_name}\n")

In [8]:
# if path already exists
if (os.path.exists(module_cache_path)) and (not overwrite_model_cache):
    # path exists and not overwriting
    if print_level > 0:
        print(f"Model already generated at {module_cache_path}\n")

In [9]:
    n_y = len(y)
    n_x = len(x)
    n = n_y + n_x
    n_p = len(p)
    assert n_p > 0  # code written to have at least one parameter
    n_ϵ = η.shape[1]
    n_z = n if Q is None else Q.shape[0]

In [10]:
    # Get the markovian variables and create substitutions
    y_subs = pd.DataFrame([make_substitutions(t, y_i) for y_i in y])
    x_subs = pd.DataFrame([make_substitutions(t, x_i) for x_i in x])
    y = y_subs['var'].values
    x = x_subs['var'].values
    y_p = y_subs['var_p'].values
    x_p = x_subs['var_p'].values
    y_ss = y_subs['var_ss'].values
    x_ss = x_subs['var_ss'].values
    subs = pd.concat([x_subs, y_subs]).reset_index(drop=True)
    all_to_markov = pd.concat([subs['markov_t'], subs['markov_tp1'], subs['markov_inf']], axis=1).values
    all_to_var = pd.concat([subs['tp1_to_var'], subs['inf_to_var']], axis=1).values

In [11]:
all_to_markov

array([[(k(t), k), (k(t + 1), k_p), (k(oo), k_ss)],
       [(z(t), z), (z(t + 1), z_p), (z(oo), z_ss)],
       [(c(t), c), (c(t + 1), c_p), (c(oo), c_ss)],
       [(q(t), q), (q(t + 1), q_p), (q(oo), q_ss)]], dtype=object)